In [37]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

from helpers import *

sys.path.append(os.path.abspath("../"))
from pax_utils import s1s2_utils

v_drift = 1.44e-4


In [38]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = "../data_lfs/merged_all_200k_1803261941.pkl"

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[nEventsTrain:][:]

df_all['intr_depth'] = v_drift*df_all['intr_drift_time'] 
#df_all['intr_depth'] = -1*v_drift*df_all['intr_drift_time'] 
#df_all['depth']      *= -1


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['x', 'y', 'depth']]
#df_train_input = df_all[:][['intr_drift_time'] + s1s2_utils.getS2integralsDataFrameColumns()]
df_train_input = df_all[:][['intr_depth'] + s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (20967, 128)
Truth shape: (20967, 3)

Input DataFrame: 


,intr_depth,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
100000,74.70288,1.707281,0.0,0.0,0.725097,0.000000,0.929653,0.748697,0.000000,0.000000,...,0.000000,0.0,3.824396,0.871452,12.733224,70.347543,21.697417,1.314791,1.191852,8.218960
100001,45.77328,2.517367,0.0,0.0,0.492983,3.377065,0.000000,2.676335,4.193884,3.148234,...,2.927112,0.0,0.000000,2.789005,4.198920,19.901871,18.400299,4.964570,6.444708,2.787059
100002,67.81680,0.000000,0.0,0.0,0.000000,0.702794,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.840159,1.323236,0.945035,1.258770,0.795450,2.138688,0.745909
100003,82.66752,0.000000,0.0,0.0,0.000000,0.000000,0.863871,3.163316,0.000000,0.000000,...,3.328040,0.0,2.893146,0.000000,0.000000,1.954019,4.644499,3.451857,0.000000,2.618479
100004,76.45680,2.984556,0.0,0.0,2.512442,0.000000,0.000000,1.233237,4.554572,2.220700,...,1.816088,0.0,5.396311,6.845104,15.127933,50.319210,27.806914,14.646062,2.901068,10.617543



Truth DataFrame:


,x,y,depth
100000,11.452923,-4.799238,74.424872
100001,19.004622,-14.180182,45.412076
100002,-0.713852,-30.766732,67.404493
100003,-31.193505,-19.178735,82.343068
100004,15.701762,-12.451200,76.135639


In [39]:

####################################################################################################
####################################################################################################

model_name     = "model_barge_xyz_elu"
folder         = "models" + "/"    
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [40]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_barge_xyz_elu.h5
Predicted



In [41]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()
arr_z_true = df_train_truth[:]['depth'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]
arr_z_pred = arr_xy_pred[:, 2]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred
arr_dz     = arr_z_true - arr_z_pred



In [42]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xyz_S2integralsDeltaT.pkl"

cols_out         = ['event_number', 'x', 'y', 'depth', 'intr_x', 'intr_y', 'intr_z', 'intr_x_nn', 'intr_y_nn']

df_out           = df_all[:][cols_out]
df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred
df_out['z_pred'] = arr_z_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,x,y,depth,intr_x,intr_y,intr_z,intr_x_nn,intr_y_nn,x_pred,y_pred,z_pred
100000,165461,11.452923,-4.799238,74.424872,10.598371,-4.364035,-74.45808,10.831001,-4.523721,11.487899,-5.553269,74.425148
100001,165462,19.004622,-14.180182,45.412076,16.832707,-14.089599,-45.52848,16.632289,-13.558204,17.562122,-14.225661,45.828938
100002,165463,-0.713852,-30.766732,67.404493,-0.623434,-27.057018,-67.57200,-1.042401,-27.421310,-0.777416,-30.569233,67.556786
100003,165468,-31.193505,-19.178735,82.343068,-27.057018,-14.338972,-82.42272,-25.929953,-13.704764,-31.346025,-18.228168,82.279259
100004,165469,15.701762,-12.451200,76.135639,13.341479,-10.348997,-76.21200,13.449440,-11.024782,15.013470,-12.433862,76.369576
